In [8]:
#read json file
import json
with open('/Users/nelesuffo/Promotion/projects/TAAT/categories/human_actions/100898.json', 'r') as file:
    data = json.load(file)

In [21]:
data['annotations'][0]

{'categoryId': 'human_actions',
 'eventId': 'location',
 'fields': {'Type of Location': 'Livingroom'},
 'id': 'location_1745396774002',
 'time': 0,
 'type': 'start',
 'videoName': '100898.MP4'}

In [23]:
# Read the JSON file
import json

with open('/Users/nelesuffo/Promotion/projects/TAAT/categories/human_actions/100898.json', 'r') as file:
    data = json.load(file)

# Extract all 'id' and 'startAnnotationId' values from the annotations
ids = {annotation['id'] for annotation in data['annotations'] if 'id' in annotation}
start_annotation_ids = {annotation['startAnnotationId'] for annotation in data['annotations'] if 'startAnnotationId' in annotation}

# Find IDs that are only in 'id' or only in 'startAnnotationId'
only_in_ids = ids - start_annotation_ids
only_in_start_annotation_ids = start_annotation_ids - ids

# Print the results
print("IDs only in 'id':", only_in_ids)
print("IDs only in 'startAnnotationId':", only_in_start_annotation_ids)

IDs only in 'id': set()
IDs only in 'startAnnotationId': set()


## Convert SuperAnnotate to TAAT format

In [3]:
import time
import uuid

def convert_annotations(input_data: dict) -> dict:
    """
    Convert annotations from a specific format to a new format.
    
    Parameters
    ---------
    input_data : dict
        The input data containing metadata and instances.
        
    Returns
    -------
    dict
        The converted output data in the new format.
    """
    video_name = input_data["metadata"]["name"]
    instances = input_data["instances"]
    
    output = {
        "video_name": video_name,
        "category_id": "human_actions",
        "annotations": []
    }

    for instance in instances:
        meta = instance.get("meta", {})
        parameters = instance.get("parameters", [])

        if meta.get("className") != "Action":
            continue

        # Skip instances explicitly marked as "input" (in case that's a label in attributes)
        attributes_check = parameters[0].get("timestamps", [{}])[0].get("attributes", [])
        if any(attr.get("name", "").lower() == "input" for attr in attributes_check):
            continue

        if not parameters or "timestamps" not in parameters[0]:
            continue

        timestamps = parameters[0]["timestamps"]
        if len(timestamps) != 2:
            continue

        start_time = timestamps[0]["timestamp"] / 1e6
        end_time = timestamps[1]["timestamp"] / 1e6

        attributes = timestamps[0].get("attributes", [])
        fields = {
            "1st Person Age Group": "",
            "1st Person Gender": "",
            "2nd Person Age Group": "",
            "2nd Person Gender": "",
            "3rd Person Age Group": "",
            "3rd Person Gender": "",
            "4th Person Age Group": "",
            "4th Person Gender": "",
            "5th Person Age Group": "",
            "5th Person Gender": "",
            "6th Person Age Group": "",
            "6th Person Gender": "",
            "Alone?": ""
        }

        event_id = None
        for attr in attributes:
            group = attr["groupName"]
            name = attr["name"]
            if group == "Type of Action":
                event_id = name.lower().replace(" ", "_")
            elif group in fields:
                fields[group] = name

        if not event_id:
            continue

        # Remove "Alone?" field for overheard speech
        if event_id == "overheard_speech":
            fields.pop("Alone?", None)

        timestamp_suffix = str(int(time.time() * 1000)) + uuid.uuid4().hex[:6]
        start_annotation_id = f"{event_id}_{timestamp_suffix}"

        output["annotations"].append({
            "categoryId": "human_actions",
            "eventId": event_id,
            "fields": fields,
            "id": start_annotation_id,
            "time": round(start_time, 6),
            "type": "start",
            "videoName": video_name
        })

        output["annotations"].append({
            "categoryId": "human_actions",
            "eventId": event_id,
            "fields": fields,
            "startAnnotationId": start_annotation_id,
            "time": round(end_time, 6),
            "type": "end",
            "videoName": video_name
        })

    return output

In [4]:
from pathlib import Path
import json

input_dir = Path('/Users/nelesuffo/Promotion/ProcessedData/annotations_superannotate')
output_dir = Path('/Users/nelesuffo/Promotion/ProcessedData/annotations_taat')

for file_path in input_dir.glob('*.json'):
    with file_path.open('r') as file:
        input_data = json.load(file)

    output_data = convert_annotations(input_data)

    # Remove '.MP4' from filename and save
    new_filename = file_path.name.replace('.MP4', '')
    output_path = output_dir / new_filename

    with output_path.open('w') as outfile:
        json.dump(output_data, outfile, indent=4)